# Advanced Lane Detection Project

## Import Packages

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

## Classes

In [2]:
# Class to store camera calibration data
class calibration():
    def __init__(self):
        self.M = []
        self.dist = []
        self.rvecs = []
        self.tvecs = []
        
# Class to store lane detection data
class detection():
    def __init__(self):
        self.detected = False
        self.fit = None
        self.fitm = None
        self.leftx = 290
        self.rightx = 990
        self.curve_rad = None
        self.off_center = None
        self.count = 0

## Calibrate Camera

In [3]:
# Load calibration image filenames
fnames = glob.glob('camera_cal/calibration*.jpg')

numx = 9 # Number of inner row corners
numy = 6 # Number of inner column corners

cal_obj_pts = []
cal_img_pts = []

# Create array of known object points
obj_pts = np.zeros((numx*numy, 3), np.float32)
obj_pts[:, :2] = np.mgrid[0:numx, 0:numy].T.reshape(-1, 2)

# Loop over calibration images
for fname in fnames:
    img = mpimg.imread(fname) # Load image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Convert image to grayscale
    ret, img_pts = cv2.findChessboardCorners(gray, (numx, numy), None) # Detect image points
    
    # If chessboard corners detected, append object and image points to those previously detected
    if ret == True: 
        cal_obj_pts.append(obj_pts)
        cal_img_pts.append(img_pts)

# Create an instance of the camera calibration data class
cal = calibration()

# Compute camera calibration matrix
ret, cal.M, cal.dist, cal.rvecs, cal.tvecs = cv2.calibrateCamera(cal_obj_pts, cal_img_pts, img.shape[0:2], None, None)

## Functions

In [4]:
# Absolute Sobel gradient threshold
def grad_thresh(img, orient, ksize, thresh):
    rch = img[:, :, 0]

    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(rch, cv2.CV_64F, 1, 0, ksize))
    
    elif orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(rch, cv2.CV_64F, 0, 1, ksize))
    
    scaled_sobel = np.uint8(abs_sobel*(255/np.max(abs_sobel)))
    
    binary = np.zeros_like(scaled_sobel)
    binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary

# Magnitude of gradient threshold
def mag_thresh(img, ksize, thresh):
    rch = img[:, :, 0]
    
    sobelx = cv2.Sobel(rch, cv2.CV_64F, 1, 0, ksize)
    sobely = cv2.Sobel(rch, cv2.CV_64F, 0, 1, ksize)
    
    mag_sobelxy = np.sqrt(sobelx**2 + sobely**2)
    
    scaled_sobel = np.uint8(mag_sobelxy*(255/np.max(mag_sobelxy)))
    
    binary = np.zeros_like(scaled_sobel)
    binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary

# Direction of gradient threshold
def dir_thresh(img, ksize, thresh):   
    rch = img[:, :, 0]
    
    abs_sobelx = np.absolute(cv2.Sobel(rch, cv2.CV_64F, 1, 0, ksize))
    abs_sobely = np.absolute(cv2.Sobel(rch, cv2.CV_64F, 0, 1, ksize))
    
    dir_sobelxy = np.arctan2(abs_sobely, abs_sobelx)
    
    binary = np.zeros_like(dir_sobelxy)
    binary[(dir_sobelxy >= thresh[0]) & (dir_sobelxy <= thresh[1])] = 1
    
    return binary

# RGB R-channel threshold
def rch_thresh(img, thresh):
    rch = img[:, :, 0]
    
    binary = np.zeros_like(rch)
    binary[(rch >= thresh[0]) & (rch <= thresh[1])] = 1
    
    return binary

# HLS S-channel threshold
def sch_thresh(img, thresh):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    sch = hls[:, :, 2]
    
    binary = np.zeros_like(sch)
    binary[(sch >= thresh[0]) & (sch <= thresh[1])] = 1
    
    return binary

# Mask image region
def mask(img):  
    # Mask vertices
    pts = np.array([[(745, 450),
                     (1130, 720),
                     (150, 720),
                     (555, 450)]],
                   dtype = np.int32)
    
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, pts, 1)
    
    masked = np.zeros_like(img)
    masked[(img == 1) & (mask ==1)] = 1
    
    return masked

# Warp image perspective
def warp(img, view):
    # Source points
    src = np.float32([[685, 450],
                     [1090, 720],
                     [190, 720],
                     [595, 450]])
    
    # Destination points
    dst = np.float32([[990, 0],
                     [990, 720],
                     [290, 720],
                     [290, 0]])
    
    # Compute transform matrix to warp image perspective into bird's eye view
    if view == 'b':
        M = cv2.getPerspectiveTransform(src, dst)
        
    # Compute transform matrix to warp image perspective into driver's view
    elif view == 'd':
        M = cv2.getPerspectiveTransform(dst, src)
    
    return cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)

# Find initial lane locations
def find_initial(img):
    histogram = np.sum(img[img.shape[0]//2:, :], axis=0)
    midpoint = histogram.shape[0]//2
    
    leftx = np.argmax(histogram[:midpoint])
    rightx = np.argmax(histogram[midpoint:]) + midpoint 

    left_fit, left_fitm = find_initial_sub(img, leftx)
    right_fit, right_fitm = find_initial_sub(img, rightx)
    
    lanes.fit = np.mean((left_fit, right_fit), axis = 0)
    lanes.fitm = np.mean((left_fitm, right_fitm), axis = 0)
    
    lanes.detected = True
    
    return None

# Find initial lane locations - sub function
def find_initial_sub(img, basex):
    num_windows = 9   
    window_height = img.shape[0]//num_windows
    margin = 40
    min_pix = 60
    xconv = 3.7/900 # Conversion from pixels to meters for x-points
    yconv = 30/720 # Conversion from pixels to meters for y-points
    
    non_zeroy = np.array(img.nonzero()[0])
    non_zerox = np.array(img.nonzero()[1])
    
    lane_inds = []
    
    # Sliding window technique
    for window in range(num_windows):
        winy_low = img.shape[0] - (window + 1)*window_height # Window lower bound
        winy_high = img.shape[0] - window*window_height # Window upper bound

        winx_left = basex - margin # Window left bound
        winx_right = basex + margin # Window right bound

        
        lane_inds_cur = ((non_zeroy >= winy_low) & (non_zeroy < winy_high) &\
                          (non_zerox >= winx_left) & (non_zerox < winx_right)).nonzero()[0]

        lane_inds.append(lane_inds_cur)
        
        # Center window on average of detected x-points
        if len(lane_inds_cur) > min_pix:
            basex = np.int(np.mean(non_zerox[lane_inds_cur]))

    lane_inds = np.concatenate(lane_inds)
    
    # Extract points using indices of non-zero pixels inside window bounds
    x = non_zerox[lane_inds]
    y = non_zeroy[lane_inds] 

    fit = [0, 0, 0]
    fitm = [0, 0, 0]
    
    # Fit polynomial to detected points
    if np.any((x, y)):
        fit = np.polyfit(y, x, 2)
        fitm = np.polyfit(y*yconv, x*xconv, 2)
    
    return fit, fitm

# Find lane locations using margin around previously found lane locations
def find_next(img, offset):
    margin = 40
    xconv = 3.7/900 # Conversion from pixels to meters for x-points
    yconv = 30/720 # Conversion from pixels to meters for y-points
    
    non_zeroy = np.array(img.nonzero()[0])
    non_zerox = np.array(img.nonzero()[1])
    
    # Detect non-zero pixels within previously computed polynomial +/- margin
    lane_inds = ((non_zerox > (lanes.fit[0]*(non_zeroy**2) + lanes.fit[1]*non_zeroy + lanes.fit[2] - margin + offset)) &\
                 (non_zerox < (lanes.fit[0]*(non_zeroy**2) + lanes.fit[1]*non_zeroy + lanes.fit[2] + margin + offset)))

    x = non_zerox[lane_inds]
    y = non_zeroy[lane_inds]
    
    fit = [0, 0, 0]
    fitm = [0, 0, 0]
    
    # Fit polynomial to detected points
    if np.any((x, y)):
        fit = np.polyfit(y, x, 2)
        fitm = np.polyfit(y*yconv, x*xconv, 2)
    
    return fit, fitm

# Calculate road curvature radius and vehicle relation to lane center
def info(shape):
    
    xconv = 3.7/900
    yconv = 30/720
    
    curve_rad = (((1 + (2*lanes.fitm[0]*shape[0]*yconv + lanes.fitm[1])**2)**1.5)/\
                 np.absolute(2*lanes.fitm[0]))
 
    if curve_rad > 1500:
        lanes.curve_rad = 0
    else:
        lanes.curve_rad = curve_rad
    
    lane_center = lanes.fit[0]*shape[0]**2 + lanes.fit[1]*shape[0] + lanes.fit[2]
    lanes.off_center = (shape[1]/2 - lane_center)*xconv
    
    return None

# Draw identified lane and road information on image
def draw(undist, warped):
    warped_zero = np.zeros_like(warped).astype(np.uint8)
    warped_color = np.dstack((warped_zero, warped_zero, warped_zero))
    
    ploty = np.linspace(0, warped.shape[0] - 1, warped.shape[0])
    
    leftx = lanes.fit[0]*ploty**2 + lanes.fit[1]*ploty + lanes.fit[2] - 350
    rightx = lanes.fit[0]*ploty**2 + lanes.fit[1]*ploty + lanes.fit[2] + 350
    
    left_pts = np.array([np.transpose(np.vstack([leftx, ploty]))])
     # Flip right points to order polygon points from top to bottom left to bottom to top right
    right_pts = np.array([np.flipud(np.transpose(np.vstack([rightx, ploty])))])
    pts = np.hstack((left_pts, right_pts))

    cv2.fillPoly(warped_color, np.int_([pts]), (0, 255, 0))
    
    # Warp image perspective into driver's view
    unwarped = warp(warped_color, view = 'd')
    combined = cv2.addWeighted(undist, 1, unwarped, 0.3, 0)
    
    curve_text = 'Curvature radius = {:d}m'.format(int(lanes.curve_rad))
    
    if lanes.off_center < 0:
        off_text = 'Vehicle {:.2f}m left of center'.format(abs(lanes.off_center))
        arrow = '>'
        arrow_pos = (1110, 620)
    
    elif lanes.off_center > 0:
        off_text = 'Vehicle {:.2f}m right of center'.format(abs(lanes.off_center))
        arrow = '<'
        arrow_pos = (100, 620)
    
    if abs(lanes.off_center) <= 0.25:
        arrow_col = (0, 255, 0)
        
    elif (abs(lanes.off_center) > 0.25) & (abs(lanes.off_center) <= 0.5):
        arrow_col = (255, 255, 0)
        
    elif abs(lanes.off_center) > 0.5:
        arrow_col = (255, 0, 0)
    
    # Draw curvature radius text on image
    cv2.putText(combined, curve_text, org = (10, 30), fontFace = cv2.FONT_HERSHEY_SIMPLEX,\
                fontScale = 1, thickness = 2, color = (0, 255, 0), bottomLeftOrigin = False)
    
    # Draw vehicle relation to lane center on image
    cv2.putText(combined, off_text, org = (10, 60), fontFace = cv2.FONT_HERSHEY_SIMPLEX,\
                       fontScale = 1, thickness = 2, color = (0, 255, 0), bottomLeftOrigin = False)
    
    # Draw turn correction arrow on image
    cv2.putText(combined, arrow, org = arrow_pos, fontFace = cv2.FONT_HERSHEY_SIMPLEX,\
                       fontScale = 4, thickness = 4, color = arrow_col)
    
    return combined

## Pipeline

In [5]:
def pipeline(img):
    
    margin = 40
    
    # Undistort image using previously computed camera calibration matrix
    undist = cv2.undistort(img, cal.M, cal.dist, None, cal.M)
    
    # Threshold undistorted image
    binary_gradx = grad_thresh(undist, orient = 'x', ksize = 3, thresh = (20, 255))
    binary_grady = grad_thresh(undist, orient = 'y', ksize = 3, thresh = (20, 255))
    
    binary_grad_mag = mag_thresh(undist, ksize = 3, thresh = (40, 255))
    binary_grad_dir = dir_thresh(undist, ksize = 3, thresh = (0.7, 1.3))
    
    binary_rch = rch_thresh(undist, thresh = (220, 255))
    binary_sch = sch_thresh(undist, thresh = (140, 255))
    
    # Combine thresholded images
    binary_comb = np.zeros_like(binary_gradx)
    binary_comb[((binary_gradx == 1) & (binary_grady == 1)) |\
                ((binary_grad_mag == 1) & (binary_grad_dir == 1)) |\
                ((binary_rch == 1) & (binary_sch == 1))] = 1
    
    # Mask image region
    masked = mask(binary_comb)
    
    # Warp image perspective into bird's eye view
    warped = warp(masked, view = 'b')
    
    # Find lane locations from scratch
    if lanes.detected == False:
        find_initial(warped)
    
    # Find lane locations using margin around previously found lane locations
    elif lanes.detected == True:
        left_fit, left_fitm = find_next(warped, -350)
        right_fit, right_fitm = find_next(warped, 350)
        
        # Average current left and right lane polynomial coefficients
        fit_cur = np.mean((left_fit, right_fit), axis = 0)
        fitm_cur = np.mean((left_fitm, right_fitm), axis = 0)

        # Calculate current left and right lane base points 
        leftx_cur = fit_cur[0]*warped.shape[0]**2 + fit_cur[1]*warped.shape[0] + fit_cur[2] - 350
        rightx_cur = fit_cur[0]*warped.shape[0]**2 + fit_cur[1]*warped.shape[0] + fit_cur[2] + 350
        
        # Compare current left and right lane base points with those previously found
        if (((leftx_cur > (lanes.leftx - margin)) & (leftx_cur < (lanes.leftx + margin))) &\
            ((rightx_cur > (lanes.rightx - margin)) & (rightx_cur < (lanes.rightx + margin)))):
            lanes.leftx = leftx_cur
            lanes.rightx = rightx_cur
            
            # Average current and previous lane polynomial coefficients
            lanes.fit = np.mean((fit_cur, lanes.fit), axis = 0)
            lanes.fitm = np.mean((fitm_cur, lanes.fitm), axis = 0)
        
        # If compared points are not within margin, find lane locations from scratch
        else:
            find_initial(warped)
    
    # Calculate road curvature radius and vehicle relation to lane center for every two video frames
    if lanes.count == 0:
        info(warped.shape)
        lanes.count = 2
    
    lanes.count = lanes.count - 1
        
    return draw(undist, warped)

## Test on Video

In [6]:
lanes = detection()

video_output = 'videos/output/project_video.mp4'
video_input = VideoFileClip('videos/input/project_video.mp4')
video_clip = video_input.fl_image(pipeline)
%time video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video videos/output/project_video.mp4
[MoviePy] Writing video videos/output/project_video.mp4


100%|█████████▉| 1260/1261 [06:02<00:00,  3.57it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/output/project_video.mp4 

CPU times: user 5min 56s, sys: 1min 16s, total: 7min 13s
Wall time: 6min 3s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))